# Plot the results of 555 digipot/ADC testing

This script generates plots using the data from the 555 digipot/ADC experiment included in this repository.

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib
matplotlib.rcParams.update({'font.size': 16, 'mathtext.default': 'regular'})
import matplotlib.pyplot as plt
import pandas

PLOT_COLUMNS = ['TRIG', 'THRES', 'OUT']

Assume we've run `digipot_adc_experiment.py` run according to the following test matrix:

\# | TRIG     | THRES    | RESET | CONT           |  OUT
---|:--------:|:--------:|:-----:|:--------------:|:-----:
 1 | **vary** | low      |  low  | V<sub>DD</sub> |
 2 | **vary** | high     |  low  | V<sub>DD</sub> |
 3 | low      | **vary** |  low  | V<sub>DD</sub> |
 4 | high     | **vary** |  low  | V<sub>DD</sub> |
 
The outputs should be saved in a series of CSV files loaded below.

In [ ]:
experiments = {
    "Experiment #1": pandas.read_csv('results/555-experiment-1.csv', skipinitialspace=True)[PLOT_COLUMNS],
    "Experiment #2": pandas.read_csv('results/555-experiment-2.csv', skipinitialspace=True)[PLOT_COLUMNS],
    "Experiment #3": pandas.read_csv('results/555-experiment-3.csv', skipinitialspace=True)[PLOT_COLUMNS],
    "Experiment #4": pandas.read_csv('results/555-experiment-4.csv', skipinitialspace=True)[PLOT_COLUMNS],
}
for title, voltages in experiments.iteritems():
    column_titles = sorted(voltages.columns, reverse=True)
    experiments[title] = voltages[column_titles]

In [ ]:
def plot_voltages(voltages, title, ax=None):
    """
    Plot all of the four voltage measurements generated by the experiment
    """
    if ax is None:
        fig, ax = plt.subplots()
        fig.set_size_inches(8, 6)
        fig.suptitle(title)
    ax.set_ylabel("Measured Volts")
    ax.set_xticks([])
    ax.set_xlabel("Elapsed Time")
    voltages.plot(ax=ax)
    
    # style lines
    lines = ax.get_lines()
    lines[0].set_linestyle('--')
    lines[0].set_linewidth(4.0)
    lines[1].set_linestyle('--')
    lines[1].set_linewidth(4.0)
    
    # modify legend
    _, labels = ax.get_legend_handles_labels()
    new_labels = ["$V_{%s}$" % label for label in labels]
    ax.legend(
        labels=new_labels,
        loc="center left",
        bbox_to_anchor=(0.00, 0.32),
        fontsize=14)
    ax.grid()

The following code is a fancy way of plotting all four data sets on a single canvas.  You can also plot the data separately, e.g.,

    for title in sorted(experiments.keys()):
        plot_voltages(experiments[title], title)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True)
fig.set_size_inches(12, 8)
fig.subplots_adjust(hspace=0.15, wspace=0.05)

for index, title in enumerate(sorted(experiments.keys())):
    voltages = experiments[title]
    icolumn = index % 2
    irow = index / 2
    ax = axes[irow, icolumn]
    plot_voltages(voltages=voltages, title=title, ax=ax)
    ax.set_title(title, fontsize=16)
    
    # turn off redundant labels
    if irow == 0:
        ax.set_xlabel("")
    if icolumn > 0:
        ax.set_ylabel("")

## Exploring hysteresis in TRIG/THRES behavior

The results from the previous section indicate there is asymmetric behavior in the relationship between TRIG and THRES.  We deduced this because experiment #2 and #3 aren't mirroring each other.

Thus, we look for hysteretical behavior by doing full high-low-high cycles.

In [ ]:
experiments = {
    "Experiment #1": pandas.read_csv('results/555-experiment-1-hysteresis.csv', skipinitialspace=True)[PLOT_COLUMNS],
    "Experiment #2": pandas.read_csv('results/555-experiment-2-hysteresis.csv', skipinitialspace=True)[PLOT_COLUMNS],
    "Experiment #3": pandas.read_csv('results/555-experiment-3-hysteresis.csv', skipinitialspace=True)[PLOT_COLUMNS],
    "Experiment #4": pandas.read_csv('results/555-experiment-4-hysteresis.csv', skipinitialspace=True)[PLOT_COLUMNS],
}
for title, voltages in experiments.iteritems():
    column_titles = sorted(voltages.columns, reverse=True)
    experiments[title] = voltages[column_titles]

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True)
fig.set_size_inches(12, 8)
fig.subplots_adjust(hspace=0.15, wspace=0.05)

for index, title in enumerate(sorted(experiments.keys())):
    voltages = experiments[title]
    icolumn = index % 2
    irow = index / 2
    ax = axes[irow, icolumn]
    plot_voltages(voltages=voltages, title=title, ax=ax)
    ax.set_title(title, fontsize=16)
    
    # turn off redundant labels
    if irow == 0:
        ax.set_xlabel("")
    if icolumn > 0:
        ax.set_ylabel("")

We also couple together TRIG and THRES to create a circuit that resembles the basis for an astable oscillator.  This shows clear hysteresis.

In [ ]:
experiment = pandas.read_csv('results/555-experiment-5.csv', skipinitialspace=True)[PLOT_COLUMNS]
plot_voltages(experiment, title="")

Now we do full wavelengths following the cycle from both high to both low by altering only one input at a time:

In [ ]:
experiment = pandas.read_csv('results/555-experiment-11.csv', skipinitialspace=True)[PLOT_COLUMNS]
plot_voltages(experiment, title="")

Going the reverse path also reveals strong hysteresis:

In [ ]:
experiment = pandas.read_csv('results/555-experiment-12.csv', skipinitialspace=True)[PLOT_COLUMNS]
plot_voltages(experiment, title="")